4つの配列(A, B, C, D)の系統樹をUPGMA法に基づいて再構築する。距離行列は以下の通りです。
可能であれば、計算をサポートするコンピュータ・ソフトウェア（任意の言語やツールを使用）を作成してください。コンピュータソフトとその計算結果、そして最終的な系統樹を示してください。図を描くのは難しいので、ソフトの計算結果をもとに手書きで系統樹を作成してもよい。

In [13]:
class Cluster():
    def __init__(self,name,numOfElements):
        #cluster name
        self.name = name
        #Number of clusters belonging to the cluster
        self.elements = numOfElements

    #Distance from other clusters   
    def set_distance(self,distance):
        self.distance = distance
        
    #Distance between clusters belonging to a cluster
    def set_distance_between_elements(self,dbe):
        self.dbe = dbe
    
    #Objects of clusters belonging to the cluster
    def set_elementsObject(self,elements):
        self.elements_object = elements


#Function to find the corresponding clusters
def search_cluster(clusters,name):
    for cluster in clusters:
        if cluster.name == name:
            return cluster
        
#Function to output the result
def print_tree(child,parent):
    if len(child.elements_object) != 0:
        if parent != "":
            print("parent :",parent.name,">>","child : ",child.name)
            print("\tHeight of contact point (",parent.dbe/2,")")
        for o in child.elements_object:
            print_tree(o,child)
    else:
        print("parent :",parent.name,">>","child : ",child.name)
        print("\tHeight of contact point (",parent.dbe/2,")")

In [14]:
class UPGMA():
            
    def search_minvalue(self,distance_matrix):
        '''
        Find the minimum value and the corresponding pair in the distance matrix.
        '''
        min_distances = []
        #Find the minimum value for each row
        for name in distance_matrix:
            distance = distance_matrix[name]
            min_distances.append([min(distance, key=distance.get),min(distance.values())])
        #Minimum value in the distance matrix
        minvalue = min([e[1] for e in min_distances])
        #Pairs corresponding to the minimum value
        pair = [e[0] for e in min_distances if e[1] ==  minvalue]
        if len(pair) == 2:
            return pair,minvalue
        else:
            print("the length of minimum value's pair is not 2")
            exit(1)
        
        
    
    def make_new_cluster(self,clusters,pair,minvalue):
        '''
        Merging two clusters to create a new cluster.
        '''
        new_cluster = []
        elements = []
        newname = "{"
        newcluster_num_of_elements = 0
        for i,p in enumerate(pair):
            e = search_cluster(clusters,p)
            newcluster_num_of_elements+=e.elements
            if i+1 == len(pair):
                newname += p + "}"
            else:
                newname += p+","
            elements.append(e)
        
        #new cluster
        c = Cluster(newname,newcluster_num_of_elements)
        #Distance between elements
        c.set_distance_between_elements(minvalue)
        #Adding an element object
        c.set_elementsObject(elements)
        clusters.append(c)
        
        return newname
    
    
    def make_new_distance_matrix(self,newname,distance_matrix,pair,clusters):
        '''
        Recalculate the distance between the new cluster and the others
        '''
        newclusters = [newname]#New cluster name group
        newcluster_distance = {}
        for name in list(distance_matrix.keys()):
            if name not in pair:
                newclusters.append(name)
                sum_distance = 0
                num_of_elements = 0
                for element in pair:
                    ec = search_cluster(clusters,element)
                    num_of_elements += ec.elements
                    #Calculating the average distance
                    sum_distance += ec.distance[name] * ec.elements
                resultdistance = sum_distance/num_of_elements
                newcluster_distance[name] = resultdistance
        search_cluster(clusters,newname).set_distance(newcluster_distance)
    
        #Creating a Distance Matrix
        distance_matrix  = {newname : newcluster_distance}
        for name1 in newclusters:
            if name1 != newname:
                newdistancemat = {}
                c = search_cluster(clusters,name1)
                for name2 in newclusters:
                    if name1 != name2:
                        if name2 == newname:#Distance to the new cluster
                            newdistancemat[newname] = newcluster_distance[name1]
                        else:#Distance from existing clusters
                            newdistancemat[name2] = c.distance[name2]
                c.set_distance(newdistancemat)
                distance_matrix[name1] = newdistancemat
                
        return distance_matrix

In [15]:
if __name__ == "__main__":

    distance_matrix = {"A":{ "B":0.1,"C":0.12,"D":0.21},
                                      "B":{"A":0.1,"C":0.04,"D":0.13},
                                      "C":{ "A":0.12,"B":0.04,"D":0.11},
                                      "D":{ "A":0.21,"B":0.13,"C":0.11}}
    all_culusters = len(distance_matrix)

    #Initialize the cluster
    clusters = []
    for name in distance_matrix :
        c = Cluster(name,1)
        c.set_distance(distance_matrix[name])
        c.set_elementsObject([])
        clusters.append(c)

    upgma=UPGMA()
    print(">>distance matrix")
    loop = 0
    while True:
        loop += 1
        #Find the minimum value in a matrix
        pair,minvalue = upgma.search_minvalue(distance_matrix)
        #Create a new cluster and get its name
        newname = upgma.make_new_cluster(clusters,pair,minvalue)
        if search_cluster(clusters,newname).elements ==all_culusters:
            break
        #Creating a Distance Matrix
        distance_matrix = upgma.make_new_distance_matrix(newname,distance_matrix,pair,clusters)
        print("loop : ",loop,"->", distance_matrix)
    print("-------------------------------")
    print(">>tree")
    print_tree(search_cluster(clusters,newname),"")

>>distance matrix
loop :  1 -> {'{C,B}': {'A': 0.11, 'D': 0.12}, 'A': {'{C,B}': 0.11, 'D': 0.21}, 'D': {'{C,B}': 0.12, 'A': 0.21}}
loop :  2 -> {'{A,{C,B}}': {'D': 0.15}, 'D': {'{A,{C,B}}': 0.15}}
-------------------------------
>>tree
parent : {D,{A,{C,B}}} >> child :  D
	Height of contact point ( 0.075 )
parent : {D,{A,{C,B}}} >> child :  {A,{C,B}}
	Height of contact point ( 0.075 )
parent : {A,{C,B}} >> child :  A
	Height of contact point ( 0.055 )
parent : {A,{C,B}} >> child :  {C,B}
	Height of contact point ( 0.055 )
parent : {C,B} >> child :  C
	Height of contact point ( 0.02 )
parent : {C,B} >> child :  B
	Height of contact point ( 0.02 )


In [16]:
distance_matrix = {"A":{ "B":0.02,"C":0.07,"D":0.09},
                                  "B":{"A":0.02,"C":0.07,"D":0.09},
                                  "C":{ "A":0.07,"B":0.07,"D":0.04},
                                  "D":{ "A":0.09,"B":0.09,"C":0.04}}
all_culusters = len(distance_matrix)

#clusterの初期化
clusters = []
for name in distance_matrix :
    c = Cluster(name,1)
    c.set_distance(distance_matrix[name])
    c.set_elementsObject([])
    clusters.append(c)

upgma=UPGMA()
loop = 0
print(">>distance matrix")
while True:
    loop += 1
    pair,minvalue = upgma.search_minvalue(distance_matrix)
    newname = upgma.make_new_cluster(clusters,pair,minvalue)
    if search_cluster(clusters,newname).elements ==all_culusters:
        break
    distance_matrix = upgma.make_new_distance_matrix(newname,distance_matrix,pair,clusters)
    print("loop : ",loop,"->", distance_matrix)
print("-------------------------------")
print(">>tree")
print_tree(search_cluster(clusters,newname),"")

>>distance matrix
loop :  1 -> {'{B,A}': {'C': 0.07, 'D': 0.09}, 'C': {'{B,A}': 0.07, 'D': 0.04}, 'D': {'{B,A}': 0.09, 'C': 0.04}}
loop :  2 -> {'{D,C}': {'{B,A}': 0.08}, '{B,A}': {'{D,C}': 0.08}}
-------------------------------
>>tree
parent : {{B,A},{D,C}} >> child :  {B,A}
	Height of contact point ( 0.04 )
parent : {B,A} >> child :  B
	Height of contact point ( 0.01 )
parent : {B,A} >> child :  A
	Height of contact point ( 0.01 )
parent : {{B,A},{D,C}} >> child :  {D,C}
	Height of contact point ( 0.04 )
parent : {D,C} >> child :  D
	Height of contact point ( 0.02 )
parent : {D,C} >> child :  C
	Height of contact point ( 0.02 )


In [46]:
distance_matrix = {"A":{ "B":0.1,"C":0.12,"D":0.21},
                                "B":{"A":0.1,"C":0.04,"D":0.13},
                                "C":{ "A":0.12,"B":0.04,"D":0.11},
                                "D":{ "A":0.21,"B":0.13,"C":0.11}}


def print_distance_matrix(distance_matrix):
    names = [name for name in distance_matrix]
    print("\n======================")
    for name in distance_matrix:
        print(name,"||",end = "")
        for n in names:
            if n not in list(distance_matrix[name].keys()):
                print(" "*5,0," |",end = "")
            else:
                print(" "*5,distance_matrix[name][n],"|",end = "")
        print("\n--------------------------------------")

In [47]:
print_distance_matrix(distance_matrix)


A ||      0  |      0.1 |      0.12 |      0.21 |
--------------------------------------
B ||      0.1 |      0  |      0.04 |      0.13 |
--------------------------------------
C ||      0.12 |      0.04 |      0  |      0.11 |
--------------------------------------
D ||      0.21 |      0.13 |      0.11 |      0  |
--------------------------------------
